<a href="https://colab.research.google.com/github/ctcerq/pyls/blob/master/logreg_gridsearch_fmri_ccl_pre_MRIfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import biblios

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold, permutation_test_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report, log_loss
from tqdm import tqdm  

Load and verify dataset

In [5]:
data=pd.read_excel('/content/Pasta4.xlsx')
data.shape

(45, 13)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   groups                45 non-null     object 
 1   l_hipp_volume         45 non-null     float64
 2   r_hipp_volume         45 non-null     float64
 3   l_hipp_meanbold       45 non-null     float64
 4   r_hipp_meanbold       45 non-null     float64
 5   l_hipp_meanbold_NvsL  45 non-null     float64
 6   r_hipp_meanbold_NvsL  45 non-null     float64
 7   mem_log               45 non-null     int64  
 8   RAVLT7RB              45 non-null     int64  
 9   groupsnum             45 non-null     int64  
 10  cclcondition          45 non-null     object 
dtypes: float64(6), int64(3), object(2)
memory usage: 4.0+ KB


In [6]:
data['groups'].unique()

array(['control', 'PIB-', 'PIB+'], dtype=object)

transform to use in some estimators

In [7]:
data.groupby('groups').describe()

l_hipp_meanbold                              ... groupsnum               
                  count      mean        std    min  ...       25%  50%  75%  max
groups                                               ...                         
PIB+               17.0  7.962471  13.137149 -24.35  ...       1.0  1.0  1.0  1.0
PIB-               14.0  7.043714   9.763331 -14.03  ...       2.0  2.0  2.0  2.0
control            14.0  8.339780  10.182454 -11.30  ...       3.0  3.0  3.0  3.0

[3 rows x 88 columns]

In [ ]:
# data['groups'] = LabelEncoder().fit_transform(data['groups'])

full model data

In [8]:
X = data.iloc[:,1:7]
y = data['groups']
print(data.iloc[0,1:7])
X.shape

l_hipp_meanbold     15.58
r_hipp_meanbold      2.07
l_hipp_volume      3429.3
r_hipp_volume      3485.9
TSD                    21
THD                     8
Name: 0, dtype: object


(45, 6)

In [9]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

nesting a pipeline for uniformization and analysis

In [10]:
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data, warm_start=True))
    ('clf', LogisticRegressionCV(cv=5, max_iter=10000, random_state=10)) #step2 - classifier
])
pipeline.steps

[('normalizer', StandardScaler()),
 ('clf', LogisticRegressionCV(cv=5, max_iter=10000, random_state=10))]

train-test model

In [ ]:
model = pipeline.fit(X, y)
y_pred = model.predict(X)
print(classification_report(y, y_pred))
print(pipeline.named_steps['clf'].coef_)
np.exp(pipeline.named_steps['clf'].coef_)

              precision    recall  f1-score   support

           C       0.62      0.57      0.59        14
           N       0.60      0.69      0.64        13
           P       0.65      0.61      0.63        18

    accuracy                           0.62        45
   macro avg       0.62      0.62      0.62        45
weighted avg       0.62      0.62      0.62        45

[[-0.75025445  1.04797632  1.01839364 -1.46529038 -0.19784007  0.55051395]
 [ 1.99600808 -1.42441717  0.37225186  0.01024548 -0.64766402  0.10733686]
 [-1.24575362  0.37644085 -1.3906455   1.45504491  0.84550409 -0.65785081]]


array([[0.47224637, 2.85187399, 2.7687436 , 0.2310109 , 0.82050106,
        1.73414406],
       [7.35961835, 0.24064868, 1.45099839, 1.01029814, 0.52326669,
        1.11330922],
       [0.28772399, 1.45708935, 0.24891458, 4.28467587, 2.32915163,
        0.51796334]])

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
#print(log_loss(y, pred, normalize=True))
#print(mean_squared_error(y, pred))
permutation_test_score(model, X, y)

KeyboardInterrupt: ignored



---



hyperparameters tunning model with set separation and gridsearch cross-validation 

In [ ]:
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = [pipeline.named_steps['clf']]
#clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(RidgeClassifier())
clfs.append(SGDClassifier(loss="log"))
clfs.append(GradientBoostingClassifier())
X_train = X # comentar linha caso use train sets
y_train = y # comentar linha caso use train sets
for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X, y)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())

NameError: ignored

In [ ]:
# define search space
from scipy.stats import loguniform
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data, warm_start=True))
    ('clf', LogisticRegression(solver='liblinear', max_iter=10000, random_state=10)) #step2 - classifier
])
space = dict()
space ['multi_class'] = ['multinomial', 'ovr']
space['solver'] = ['lbfgs', 'newton-cg', 'sag', 'saga']
space['penalty'] = ['none', 'l1', 'l2']
space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
cv = RepeatedStratifiedKFold(random_state=1)
search = GridSearchCV(pipeline.named_steps['clf'], space, cv=cv, scoring='accuracy', n_jobs=-1, refit=True)
result = search.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2400 fits failed out of a total of 9600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
800 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: S

In [ ]:
y_pred = search.predict(X)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.3688888888888889
Best Hyperparameters: {'C': 0.001, 'multi_class': 'multinomial', 'penalty': 'l1', 'solver': 'saga'}


In [ ]:
print(classification_report(y, y_pred))
metrics = pd.DataFrame()

              precision    recall  f1-score   support

         con       0.62      0.53      0.57        15
         neg       0.50      0.57      0.53        14
         pos       0.59      0.59      0.59        17

    accuracy                           0.57        46
   macro avg       0.57      0.56      0.56        46
weighted avg       0.57      0.57      0.57        46



fitting and analysing the best hyperparameters


In [ ]:
coefs = np.exp(pd.DataFrame(pipeline.named_steps['clf'].coef_))
coefs.to_excel('coefsccl.xlsx')
print(coefs)
print(classification_report(y, y_pred))
model.classes_

AttributeError: ignored

In [ ]:
permutation_test_score(model, X, y)

necessário verificar

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
pipeline.set_params(clf= SVC())
pipeline.steps

[('normalizer', StandardScaler()), ('clf', SVC())]

In [ ]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'normalizer', 'clf', 'normalizer__copy', 'normalizer__with_mean', 'normalizer__with_std', 'clf__C', 'clf__break_ties', 'clf__cache_size', 'clf__class_weight', 'clf__coef0', 'clf__decision_function_shape', 'clf__degree', 'clf__gamma', 'clf__kernel', 'clf__max_iter', 'clf__probability', 'clf__random_state', 'clf__shrinking', 'clf__tol', 'clf__verbose'])

In [ ]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    'clf__kernel' : ['linear', 'rbf'],
    'clf__C' : np.linspace(0.1,1.2,12)
})
search = cv_grid.fit(X, y)

ValueError: ignored

In [ ]:
clf.parameters

In [ ]:
tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
]
scores = ["precision", "recall"]
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    clf = GridSearchCV(SVC(), tuned_parameters, scoring="%s_macro" % score)
    clf.fit(X_train, y_train)

In [ ]:
  cv_grid.best_params_

{'clf__C': 1.0999999999999999, 'clf__kernel': 'rbf'}

In [ ]:
y_pred = cv_grid.predict(X)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, y_pred)
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))
print(classification_report(y, y_pred))

Accuracy of the best classifier after CV is 75.556%
              precision    recall  f1-score   support

           C       0.73      0.79      0.76        14
           N       0.88      0.54      0.67        13
           P       0.73      0.89      0.80        18

    accuracy                           0.76        45
   macro avg       0.78      0.74      0.74        45
weighted avg       0.77      0.76      0.75        45



In [ ]:
cv_grid.get_params

<bound method BaseEstimator.get_params of GridSearchCV(estimator=Pipeline(steps=[('normalizer', StandardScaler()),
                                       ('clf', SVC(probability=True))]),
             param_grid={'clf__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2]),
                         'clf__kernel': ['linear', 'rbf']})>

In [ ]:
KNN_model_default = KNeighborsClassifier()
KNN_model_default.fit(X, y)
y_pred_KNN_default =KNN_model_default.predict(X)
parameters_KNN = {
    'n_neighbors': (1,5, 10),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')
}
grid_search_KNN = GridSearchCV(
    estimator=KNN_model_default,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    cv = 3)
KNN_1=grid_search_KNN.fit(X, y)
y_pred_KNN1 =KNN_1.predict(X)

In [ ]:
grid_search_KNN.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': (20, 40, 1),
                         'metric': ('minkowski', 'chebyshev'),
                         'n_neighbors': (1, 5, 10), 'p': (1, 2),
                         'weights': ('uniform', 'distance')},
             scoring='accuracy')>

In [ ]:
KNN_1.best_score_


0.6222222222222222

In [ ]:

print(classification_report(y, y_pred_KNN1))

              precision    recall  f1-score   support

           C       0.57      0.86      0.69        14
           N       0.62      0.38      0.48        13
           P       0.81      0.72      0.76        18

    accuracy                           0.67        45
   macro avg       0.67      0.65      0.64        45
weighted avg       0.68      0.67      0.66        45



In [ ]:
from sklearn.linear_model import RidgeClassifierCV
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data, warm_start=True))
    ('clf', RidgeClassifierCV(normalize=True, alphas=np.logspace(-6, 6, 13), cv=cv)) #step2 - classifier
])
pipeline.steps


[('normalizer', StandardScaler()),
 ('clf',
  RidgeClassifierCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
         1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
                    cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
                    normalize=True))]

In [ ]:
model = pipeline.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), RidgeClassifier())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from skl

In [ ]:
model.named_steps.clf.coef_

array([[ 0.0328886 ,  0.07275447,  0.03182987, -0.08972389,  0.06912308,
         0.03409587],
       [ 0.14499951,  0.01412449, -0.00468487,  0.02874627, -0.1081301 ,
        -0.06355871],
       [-0.17788811, -0.08687896, -0.027145  ,  0.06097762,  0.03900701,
         0.02946284]])

In [ ]:
y_pred = model.predict(X)

In [ ]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           C       0.57      0.29      0.38        14
           N       0.55      0.46      0.50        13
           P       0.56      0.83      0.67        18

    accuracy                           0.56        45
   macro avg       0.56      0.53      0.52        45
weighted avg       0.56      0.56      0.53        45

